# Часть 2: **Парсинг web-страниц**
Автор: `Ильин Матвей 23712`

### Описание:
В качестве примера для реализации алгоритма парсинга был выбран новостной сайт: https://ngs.ru/ и библиотека BeautifulSoup4.


### Импортирование необходимых библиотек

In [1]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests
import pandas as pd

### Проверка соединения с сервером

In [11]:
# Укажите дату, начиная с которой будут парситься новости с сайта НГС
start_date = '01.03.2024'
# Укажите дату, до которой будут парситься новости с сайта НГС
end_date = '01.05.2024'


url = f'https://ngs.ru/text/?dateFrom={start_date}&dateTo={end_date}'
response = requests.get(url)
print(response)

<Response [200]>


### Алгоритм парсинга

In [12]:
bs = BeautifulSoup(response.text,"lxml")
page_amount = int(bs.find_all('a', 'KrYc4')[-1].text)
print(f"Количество страниц: {page_amount}")

articles_list = []
dates_list = []
views_amount_list = []
comments_amount_list = []


for page in tqdm(range(1, page_amount+1)):
    url = f'https://ngs.ru/text/?dateFrom={start_date}&dateTo={end_date}&page={page}'
    response = requests.get(url)
    bs = BeautifulSoup(response.text,"lxml")
    
    all_news = bs.find_all('div', 'eQ4k7')
    for news in all_news:
        article = news.find('h2', 'h9Jmx').text
        articles_list.append(article)
        date = news.find('div', 'Hiu4B vx3Rq').find('time', class_='_2DfZq')['datetime']
        dates_list.append(date)
        info = news.find_all('span', '_3mETe')
        try:
            views_amount_list.append(info[0].text)
        except:
            views_amount_list.append(0)
        try:
            comments_amount_list.append(info[1].text)
        except:
            comments_amount_list.append(0)

df_news = pd.DataFrame({'arcticle': articles_list,
                        'date': dates_list,
                        'views_amount': views_amount_list,
                        'comments_amount': comments_amount_list})


Количество страниц: 98


100%|██████████| 98/98 [01:25<00:00,  1.14it/s]


### Проверка результатов парсинга

In [13]:
df_news

,arcticle,date,views_amount,comments_amount
0,"Вакцина от COVID-19 могла вызывать тромбоз, из...",2024-05-01T23:48:39,8 726,149
1,Пик метеорного потока Эта-Аквариды увидят ново...,2024-05-01T23:45:00,3 133,8
2,В Новосибирске отменили концерт Леши Свика — п...,2024-05-01T22:50:00,7 378,36
3,Порывы до 21 м/с: в Новосибирской области объя...,2024-05-01T21:20:00,9 616,23
4,"«Врачи не верили»: история россиянки, 18 лет с...",2024-05-01T20:30:00,7 023,19
...,...,...,...,...
3886,Голова белого медвежонка застряла в конусе — з...,2024-03-01T00:16:00,6 551,10
3887,Современная Италия в центре Новосибирска: неда...,2024-03-01T00:15:00,1 460,0
3888,Изменит облик Новосибирска: руководитель извес...,2024-03-01T00:00:00,22 148,0
3889,"Александр Малинин, Павел Деревянко, Борис Андр...",2024-03-01T00:00:00,6 447,0
